In [1]:
%matplotlib inline
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import MNIST

from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, ConcatDataset

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
##############################################################
# CLASS TO NORMALIZE OUR DATA
##############################################################

class Normalization(object):
    def __init__(self, mean, std):
        self.mean = mean.view(-1, 1, 1)
        self.std = std.view(-1, 1, 1)
    def __call__(self, sample):
        image, label1, label2, label3 = sample['image'],\
        sample['label_age'], sample['label_gender'], sample['label_race']
        
        return {'image': image,
                'label_age': label1,
                'label_gender': label2,
                'label_race': label3}

##############################################################
# CLASS FOR DATA HANDLING IN PYTORCH
##############################################################

class MyData(Dataset):
    def __init__(self, dataset_tensor, train=True, transform=None):
        
        #Leaving only image related columns
        features=dataset_tensor[:,:39999]
     
        ##########################################################
        # Setting labels
        ##########################################################
       
        label_chest_circumference=dataset_tensor[:,40000]
        label_height=dataset_tensor[:,40001]
        label_inseam=dataset_tensor[:,40002]
        label_left_arm_length=dataset_tensor[:,40003]
        label_pelvis_circumference=dataset_tensor[:,40004]
        label_right_arm_length=dataset_tensor[:,40005]
        label_shoulder_width=dataset_tensor[:,40006]
        label_waist_circumference=dataset_tensor[:,40007]
        
        ##########################################################
        # splitting the data into train and validation set
        ##########################################################
        
        X_train, X_test, y_chest_circumference_train, y_chest_circumference_test,\
        y_height_train, y_height_test, y_inseam_train, y_inseam_test,\
        y_left_arm_length_train, y_left_arm_length_test,\
        y_pelvis_circumference_train, y_pelvis_circumference_test,\
        y_right_arm_length_train, y_right_arm_length_test,\
        y_shoulder_width_train, y_shoulder_width_test,\
        y_waist_circumference_train, y_waist_circumference_test = train_test_split(features, label_chest_circumference, 
                           label_height, label_inseam,
                           label_left_arm_length, label_pelvis_circumference,
                           label_right_arm_length, label_shoulder_width,
                           label_waist_circumference, test_size=0.2)
        
        if train==True:
            self.x=X_train
            self.chest_circumference_y=y_chest_circumference_train
            self.height_y=y_height_train
            self.inseam_y=y_inseam_train
            self.left_arm_length_y=y_left_arm_length_train
            self.pelvis_circumference_y=y_pelvis_circumference_train
            self.right_arm_length_y=y_right_arm_length_train
            self.shoulder_width_y=y_shoulder_width_train
            self.waist_circumference_y=y_waist_circumference_train
        else:
            self.x=X_test
            self.chest_circumference_y=y_chest_circumference_test
            self.height_y=y_height_test
            self.inseam_y=y_inseam_test
            self.left_arm_length_y=y_left_arm_length_test
            self.pelvis_circumference_y=y_pelvis_circumference_test
            self.right_arm_length_y=y_right_arm_length_test
            self.shoulder_width_y=y_shoulder_width_test
            self.waist_circumference_y=y_waist_circumference_test 
            
        #############################
        # TRANSFORMS
        #############################
        
        # normalize data, w.o. Bessel Correction -> n instead of n-1
        std, mean = torch.std_mean(self.x, unbiased=False)
        
        if transform is None:
            intrinsic_transform = torch.nn.Sequential(transforms.Normalize(mean, std))
        
            #Applying transformation
            self.transform=intrinsic_transform
        else:
            #Applying transformation
            self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=torch.tensor(self.x[idx, 0:])
        label1=torch.tensor([self.chest_circumference_y[idx]]).float()
        label2=torch.tensor([self.height_y[idx]]).float()
        label3=torch.tensor([self.inseam_y[idx]]).float()
        label4=torch.tensor([self.left_arm_length_y[idx]]).float()
        label5=torch.tensor([self.pelvis_circumference_y[idx]]).float()
        label6=torch.tensor([self.right_arm_length_y[idx]]).float()
        label7=torch.tensor([self.shoulder_width_y[idx]]).float()
        label8=torch.tensor([self.waist_circumference_y[idx]]).float()
        labels=torch.cat((label1, label2, label3, label4, label5, label6, label7, label8), -1)
        
        #Applying transformation
        #if self.transform:
            #image=self.transform(image)
            
        return (image,labels)

In [3]:
##############################################################
# LOAD TENSORS FROM FILES
##############################################################

device = torch.device('cuda')

plain_tensor = torch.load('tensors/humans_monsters_plain_full_tensor.pt', map_location=device)
#rgb_tensor = torch.load('tensors/humans_monsters_rgb_full_tensor.pt', map_location=device)
#texture_tensor = torch.load('tensors/humans_monsters_texture_full_tensor.pt', map_location=device)

##############################################################
# CREATE DATA OBJECTS AND PREPARE THEM FOR THE CNN WITH
# THE DATA LOADER CLASS
##############################################################

plain_dataset_train = MyData(plain_tensor, train=True)
plain_dataset_test = MyData(plain_tensor, train=False)

#rgb_dataset = MyData(rgb_tensor)
#texture_dataset = MyData(texture_tensor)
                       
test_loader = DataLoader(plain_dataset_test, batch_size=10, num_workers=0)
train_loader = DataLoader(plain_dataset_train, batch_size=10, num_workers=0)
#loader = DataLoader(plain_dataset, batch_size=10, num_workers=0, shuffle=False)
#loader = DataLoader(plain_dataset, batch_size=10, num_workers=0, shuffle=False)
                       
print("Dataset/s loaded")

Dataset/s loaded


#### Original Architecture

* Input Layer: fixed image of size 200 x 200 x 1
* Second Layer: Convolution with 5-pixels square kernel. Output: feature map of size 196 x 196 x 8.

* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Pooling Layer: Max pooling with stride 2 
* Convolution Layer: Convolution with 5-pixels square kernel and 16 output channels Output: Tensor of size 94 x 94 x 16.

* Pooling Layer: Max pooling with stride 2. And then flatten. Output: tensor of size 35344

* Fully Connected Layer:
* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Regressor Layer: Output: 8 human body dimensions in meters

#### Training
* 20 epochs
* mini_batc_size = 100
* Loss = MSE
* learning_rate = 0.01
* momentum = 0.9
* TODO: regressor funktion
* For layers
* 
* https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html

In [4]:
class ConvNet(nn.Module):
    '''
    Simple Convolutional Neural Network
    '''
    def __init__(self):
        
        super().__init__()
        self.layers = nn.Sequential(

          # input layer 200 x 200 x 1 - (in: 1 color channel, out: 8 Channels, ...)
          # output 196 x 196 x 8  
          nn.Conv2d(1, 8, kernel_size=5),
          nn.ReLU(),

          # input 196 x 196 x 8 
          # output 98 x 98 x 8
          nn.MaxPool2d(stride=2, kernel_size=1),

          # input 98 x 98 x 8
          # output 94 x 94 x 16
          nn.Conv2d(8, 16, kernel_size=5),
          nn.ReLU(),

          # input 94 x 94 x 16
          # output 47 x 47 x 16
          nn.MaxPool2d(stride=2, kernel_size=1),

          # Pooling Layer: Max pooling with stride 2. And then flatten.
          # Output: tensor of size 35344 = 47 x 47 x 16
          nn.Flatten(),
          nn.Linear(35344, 35344),
          nn.ReLU(),

          # Last Layer - Regressor
          # input 35344
          # output 8
          nn.Linear(35344, 8)

        )

    def forward(self, x):
        inputs = x.unsqueeze(0)
        outputs = self.layers(inputs)
        return outputs

In [5]:
loss_fn = nn.MSELoss()
model = ConvNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [6]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [7]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(test_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


C:\Users\nero\AppData\Local\Temp/ipykernel_21416/362273492.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image=torch.tensor(self.x[idx, 0:])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [8, 1, 5, 5], but got 3-dimensional input of size [1, 10, 39999] instead